# Yolov5

In [1]:
!pip install torch torchvision


In [2]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 16450, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 16450 (delta 17), reused 29 (delta 11), pack-reused 16408
Receiving objects: 100% (16450/16450), 14.96 MiB | 15.89 MiB/s, done.
Resolving deltas: 100% (11282/11282), done.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install content

  Preparing metadata (setup.py) ... done
  Created wheel for content: filename=content-0.0.1-py3-none-any.whl size=1381 sha256=4f8fa41e39365251652875612891bbc95b7d6600a6785dbca18d8e92f0dbe92c
  Stored in directory: /root/.cache/pip/wheels/83/f5/a5/76291548fc032f673b49b2c93a9e5b8c43c8c1c5706cb02f15
Successfully built content


In [5]:
!pip show content

Name: content
Version: 0.0.1
Summary: Nothing
Home-page: https://github.com/eaybek/content
Author: Erdem Aybek
Author-email: eaybek@gmail.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: 


In [6]:
import sys
print(sys.path)

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython']


In [7]:
import sys
sys.path.append("/path/to/module/directory")

In [8]:
import content

In [9]:
import importlib
try:
    importlib.import_module('content')
except ModuleNotFoundError as e:
    print(f"Module 'content' not found: {e}")

In [10]:
import os
import cv2
import json
import glob
import torch
import shutil
import random
import requests
import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt
from drive.MyDrive.utility.src.create_annotations import *
import matplotlib.patches as patches
from shapely.geometry import Polygon, MultiPolygon


In [11]:


source_dir = '/content/drive/MyDrive/utility/dataset/input_images'
output_dir = '/content/drive/MyDrive/utility/Yolov5_dataset'
train_dir = '/content/drive/MyDrive/utility/Yolov5_dataset/train'
test_dir = '/content/drive/MyDrive/utility/Yolov5_dataset/test'
val_dir = '/content/drive/MyDrive/utility/Yolov5_dataset/val'

split_percentage_train = 0.8
split_percentage_test = 0.1

# Create output, train, test, and val directories if they don't exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Get a list of all the image files
files = os.listdir(source_dir)
random.seed(123)  # Set a fixed random seed
random.shuffle(files)  # Shuffle the files for random splitting

# Calculate the number of files for each split
num_train = int(len(files) * split_percentage_train)
num_test = int(len(files) * split_percentage_test)
num_val = len(files) - num_train - num_test

# Split the files into train, test, and val sets
train_files = files[:num_train]
test_files = files[num_train:num_train + num_test]
val_files = files[num_train + num_test:]

# Copy files to the respective directories
for file in train_files:
    shutil.copy(os.path.join(source_dir, file), os.path.join(train_dir, file))
for file in test_files:
    shutil.copy(os.path.join(source_dir, file), os.path.join(test_dir, file))
for file in val_files:
    shutil.copy(os.path.join(source_dir, file), os.path.join(val_dir, file))


In [12]:
def coco_to_yolo(coco_annotation_path, output_dir):
    with open(coco_annotation_path, 'r') as f:
        coco_annotations = json.load(f)

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    for image_info in coco_annotations['images']:
        img_id = image_info['id']
        img_name = image_info['file_name']
        img_annotations = [ann for ann in coco_annotations['annotations'] if ann['image_id'] == img_id]

        yolo_annotations = []
        for ann in img_annotations:
            bbox = ann['bbox']
            x_center = bbox[0] + bbox[2] / 2
            y_center = bbox[1] + bbox[3] / 2
            width = bbox[2]
            height = bbox[3]

            yolo_annotations.append(f"{ann['category_id']} {x_center} {y_center} {width} {height}")

        with open(os.path.join(output_dir, os.path.splitext(img_name)[0] + '.txt'), 'w') as f:
            f.write('\n'.join(yolo_annotations))

coco_annotation_path = '/content/drive/MyDrive/utility/dataset/output/input_images.json'
output_dir = '/content/drive/MyDrive/utility/Yolov5_dataset/yolo'
coco_to_yolo(coco_annotation_path, output_dir)


In [13]:


model = torch.hub.load('ultralytics/yolov5', 'yolov5l', pretrained=True)
# Path to the directory containing train, test, and val folders
data_dir = "/content/drive/MyDrive/utility/Yolov5_dataset"

# Perform inference on each set (train, test, val)
for set_name in ['train', 'test', 'val']:
    image_dir = Path(data_dir) / set_name
    annotation_dir = Path(data_dir) / 'yolo' / set_name

    # List of image files
    image_files = [file for file in image_dir.rglob('*.jpg')]

    # Perform inference on each image
    for image_file in image_files:
        # Load image
        img = Image.open(image_file)

        # Get corresponding annotation file
        annotation_file = annotation_dir / (image_file.stem + '.txt')

        # Perform inference
        results = model(img)

        # Process results (e.g., apply masking using annotations)



/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


requirements: Ultralytics requirement ['gitpython>=3.1.30'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 51.7 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 5.1s, installed 1 package: ['gitpython>=3.1.30']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-2-24 Python-3.10.12 torch-2.1.0+cu121 CPU

100%|██████████| 89.3M/89.3M [00:00<00:00, 341MB/s]

Fusing layers... 
YOLOv5l summary: 367 layers, 46533693 parameters, 0 gradients, 109.0 GFLOPs
Adding AutoShape... 


In [14]:
results.print()

image 1/1: 450x600 1 pizza
Speed: 6.7ms pre-process, 1087.4ms inference, 0.7ms NMS per image at shape (1, 3, 480, 640)


In [15]:
data_dir = "/content/drive/MyDrive/utility/Yolov5_dataset"
image_dir = Path(data_dir)
annotation_dir = Path(data_dir) / 'yolo'


In [16]:
def infer_and_plot(image_file, set_name):
    """
    Performs inference on an image and plots the predictions.

    Args:
        image_file (Path): Path to the image file.
        set_name (str): Name of the image set (train, test, or val).
    """

    # Load image
    img = Image.open(image_file)

    # Perform inference
    results = model(img)

    # Extract predicted bounding boxes and labels
    boxes = results.pandas().xyxy[0]  # Assuming single image inference and batch size 1

    # Convert bounding boxes to NumPy array for easier manipulation
    boxes_np = boxes[['xmin', 'ymin', 'xmax', 'ymax']].to_numpy()

    # Create a copy of the image for visualization
    img_copy = np.array(img)

    # Draw bounding boxes on the image copy
    for box in boxes_np:
        xmin, ymin, xmax, ymax = box
        cv2.rectangle(img_copy, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)

    # Display the image with predictions
    plt.figure(figsize=(10, 6))
    plt.imshow(img_copy)
    plt.title(f"Predictions on {set_name} image: {image_file.name}")
    plt.axis('off')
    plt.show()
    plt.close()

for set_name in ['train', 'test', 'val']:
    image_dir_set = image_dir / set_name

    # List image files in the current set
    image_files = [file for file in image_dir_set.rglob('*.jpg')]

    # Process each image and plot predictions
    for image_file in image_files:
        infer_and_plot(image_file, set_name)


In [17]:
for set_name in ['train', 'test', 'val']:
    image_dir_set = image_dir / set_name

    # List image files in the current set
    image_files = [file for file in image_dir_set.rglob('*.jpg')]

    # Process each image and plot predictions
    for image_file in image_files:
        infer_and_plot(image_file, set_name)
